#  EPA-122A *Spatial* Data Science


## Lab 1 - part 3: Web Scraping

**TU Delft**<br>
**Q2 2024**<br>
**Instructors:** Giacomo Marangoni, Theodoros Chatzivasileiadis <br>


---


In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import time
import os
from IPython.display import Image

# **NOTE:** The contents of this notebook are advanced in nature and only optional to look at.

## Table of Contents

* [Learning Goals](#section0)
* [Introduction to Web Servers and HTTP](#section1)
* [Download webpages and get basic properties](#section2)
* [Beautiful Soup](#section3)
    * [Parse the page with Beautiful Soup](#section3_1)
    * [Takeaway lesson](#section3_2)
* [String formatting](#section4)
* [Walkthrough Example of web scraping](#section5)
* [How To Utilize APIs](#section6)
    * [What is an API?](#section6_1)
    * [Making API Requests in Python](#section6_2)
    * [First API Request](#section6_3)

## Learning Goals <a class="anchor" id="section0"></a>

- Understand the structure of a web page
- Understand how to use Beautiful soup to scrape content from web pages.
- Feel comfortable storing and manipulating the content in various formats.
- Understand how to convert structured format into a Pandas DataFrame

In this lab, we'll scrape Projects Gutenberg's Best Books list:

https://www.gutenberg.org/ebooks/bookshelf/13?start_index=1 .

We'll walk through scraping the list pages for the book names/urls. First, we start with an even simpler example.

## Introduction to Web Servers and HTTP <a class="anchor" id="section1"></a>

A web server is just a computer -- usually a powerful one, but ultimately it's just another computer -- that runs a long/continuous process that listens for requests on a pre-specified (Internet) _port_ on your computer. It responds to those requests via a protocol called HTTP (HyperText Transfer Protocol). HTTPS is the secure version. When we use a web browser and navigate to a web page, our browser is actually sending a request on our behalf to a specific web server. The browser request is essentially saying "hey, please give me the web page contents", and it's up to the browser to correctly render that raw content into a coherent manner, dependent on the format of the file. For example, HTML is one format, XML is another format, and so on.

Ideally (and usually), the web server complies with the request and all is fine. As part of this communication exchange with web servers, the server also sends a status code.
- If the code starts with a **2**, it means the request was successful.
- If the code starts with a **4**, it means there was a client error (you, as the user, are the client). For example, ever receive a 404 File Not Found error because a web page doesn't exist? This is an example of a client error, because you are requesting a bogus item.
- If the code starts with a **5**, it means there was a server error (often that your request was incorrectly formed).

[Click here](https://www.restapitutorial.com/httpstatuscodes.html) for a full list of status codes.

As an analogy, you can think of a web server as being like a server at a restaurant; its goal is _serve_ you your requests. When you try to order something not on the menu (i.e., ask for a web page at a wrong location), the server says 'sorry, we don't have that' (i.e., 404, client error; your mistake).

**IMPORTANT:**
As humans, we visit pages in a sane, reasonable rate. However, as we start to scrape web pages with our computers, we will be sending requests with our code, and thus, we can make requests at an incredible rate. This is potentially dangerous because it's akin to going to a restaurant and bombarding the server(s) with thousands of food orders. Very often, the restaurant will ban you (i.e., Harvard's network gets banned from the website, and you are potentially held responsible in some capacity?). It is imperative to be responsible and careful. In fact, this act of flooding web pages with requests is the single-most popular, yet archiac, method for maliciously attacking websites / computers with Internet connections. In short, be respectful and careful with your decisions and code. It is better to err on the side of caution, which includes using the **``time.sleep()`` function** to pause your code's execution between subsequent requests. ``time.sleep(2)`` should be fine when making just a few dozen requests. Each site has its own rules, which are often visible via their site's ``robots.txt`` file.

### Additional Resources

**HTML:** if you are not familiar with HTML see https://www.w3schools.com/html/ or one of the many tutorials on the internet.

**Document Object Model (DOM):** for more on this programming interface for HTML and XML documents see https://www.w3schools.com/js/js_htmldom.asp.

# Download webpages and get basic properties <a class="anchor" id="section2"></a>

``Requests`` is a highly useful Python library that allows us to fetch web pages.
``BeautifulSoup`` is a phenomenal Python library that allows us to easily parse web content and perform basic extraction.

If one wishes to scrape webpages, one usually uses ``requests`` to fetch the page and ``BeautifulSoup`` to parse the page's meaningful components. Webpages can be messy, despite having a structured format, which is why BeautifulSoup is so handy.

Let's get started:

In [ ]:
from bs4 import BeautifulSoup
import requests

To fetch a webpage's content, we can simply use the ``get()`` function within the requests library:

In [ ]:
url = "https://www.npr.org/2018/11/05/664395755/what-if-the-polls-are-wrong-again-4-scenarios-for-what-might-happen-in-the-elect"
response = requests.get(url)  # you can use any URL that you wish

The response variable has many highly useful attributes, such as:
- status_code
- text
- content

Let's try each of them!

### response.status_code

In [ ]:
response.status_code

You should have received a status code of 200, which means the page was successfully found on the server and sent to receiver (aka client/user/you). [Again, you can click here](https://www.restapitutorial.com/httpstatuscodes.html) for a full list of status codes.

### response.text


In [ ]:
response.text

Holy moly! That looks awful. If we use our browser to visit the URL, then right-click the page and click 'View Page Source', we see that it is identical to this chunk of glorious text.

### response.content

In [ ]:
response.content

What?! This seems identical to the ``.text`` field. However, the careful eye would notice that the very 1st characters differ; that is, ``.content`` has a *b'* character at the beginning, which in Python syntax denotes that the data type is bytes, whereas the ``.text`` field did not have it and is a regular String.

Ok, so that's great, but how do we make sense of this text? We could manually parse it, but that's tedious and difficult. As mentioned, BeautifulSoup is specifically designed to parse this exact content (any webpage content).

## Beautiful Soup <a class="anchor" id="section3"></a>

The [documentation for BeautifulSoup is found here](https://www.crummy.com/software/BeautifulSoup/bs4/doc/).

A BeautifulSoup object can be initialized with the ``.content`` from request and a flag denoting the type of parser that we should use. For example, we could specify ``html.parser``, ``lxml``, etc [documentation here](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#differences-between-parsers). Since we are interested in standard webpages that use HTML, let's specify the html.parser:

In [ ]:
soup = BeautifulSoup(response.content, "html.parser")
soup

Alright! That looks a little better; there's some whitespace formatting, adding some structure to our content! HTML code is structured by `<tags>`. Every tag has an opening and closing portion, denoted by ``< >`` and ``</ >``, respectively. If we want just the text (not the tags), we can use:

In [ ]:
soup.get_text()

There's some tricky Javascript still nesting within it, but it definitely cleaned up a bit. On other websites, you may find even clearer text extraction.

As detailed in the [BeautifulSoup documentation](https://www.crummy.com/software/BeautifulSoup/bs4/doc/), the easiest way to navigate through the tags is to simply name the tag you're interested in. For example:

In [ ]:
soup.head  # fetches the head tag, which ecompasses the title tag

Usually head tags are small and only contain the most important contents; however, here, there's some Javascript code. The ``title`` tag resides within the head tag.

In [ ]:
soup.title  # we can specifically call for the title tag

This result includes the tag itself. To get just the text within the tags, we can use the ``.name`` property.

In [ ]:
soup.title.string

We can navigate to the parent tag (the tag that encompasses the current tag) via the ``.parent`` attribute:

In [ ]:
soup.title.parent.name

### Parse the page with Beautiful Soup <a class="anchor" id="section3_1"></a>
In HTML code, paragraphs are often denoated with a ``<p>`` tag.

In [ ]:
soup.p

This returns the first paragraph, and we can access properties of the given tag with the same syntax we use for dictionaries and dataframes:

In [ ]:
c = BeautifulSoup('<p class="body"></p>')
c.p["class"]

In [ ]:
c.p.attrs

In addition to 'paragraph' (aka p) tags, link tags are also very common and are denoted by ``<a>`` tags

In [ ]:
soup.a

It is called the a tag because links are also called 'anchors'. Nearly every page has multiple paragraphs and anchors, so how do we access the subsequent tags? There are two common functions, `.find()` and `.find_all()`.

In [ ]:
soup.find("title")

In [ ]:
soup.find_all("title")

Here, the results were seemingly the same, since there is only one title to a webpage. However, you'll notice that ``.find_all()`` returned a list, not a single item. Sure, there was only one item in the list, but it returned a list. As the name implies, find_all() returns all items that match the passed-in tag.

In [ ]:
soup.find_all("a")

Look at all of those links! Amazing. It might be hard to read but the **href** portion of an *a* tag denotes the URL, and we can capture it via the ``.get()`` function.

In [ ]:
for link in soup.find_all(
    "a"
):  # we could optionally pass the href=True flag .find_all('a', href=True)
    print(link.get("href"))

Many of those links are relative to the current URL (e.g., /section/news/).

In [ ]:
paragraphs = soup.find_all("p")
paragraphs

If we want just the paragraph text:

In [ ]:
for pa in paragraphs:
    print(pa.get_text())

Since there are multiple tags and various attributes, it is useful to check the data type of BeautifulSoup objects:

In [ ]:
type(soup.find("p"))

Since the ``.find()`` function returns a BeautifulSoup element, we can tack on multiple calls that continue to return elements:

In [ ]:
soup.find("p")

In [ ]:
soup.find("p").find("a")

In [ ]:
soup.find("p").find("a").attrs["href"]  # att

In [ ]:
soup.find("p").find("a").text

That doesn't look pretty, but it makes sense because if you look at what ``.find('a')`` returned, there is plenty of whitespace. We can remove that with Python's built-in ``.strip()`` function.

In [ ]:
soup.find("p").find("a").text.strip()

**NOTE:** above, we accessed the attributes of a link by using the property ``.attrs``. ``.attrs`` takes a dictionary as a parameter, and in the example above, we only provided the _key_, not a _value_, too. That is, we only cared that the ``<a>`` tag had an attribute named ``href`` (which we grabbed by typing that command), and we made no specific demands on what the value must be. In other words, regardless of the value of _href_, we grabbed that element. Alternatively, if you inspect your HTML code and notice select regions for which you'd like to extract text, you can specify it as part of the attributes, too!

For example, in the full ``response.text``, we see the following line:

``<header class="npr-header" id="globalheader" aria-label="NPR header">``

Let's say that we know that the information we care about is within tags that match this template (i.e., **class** is an attribute, and its value is **'npr-header'**).

In [ ]:
# soup.find('header') has 1 attribute called "role"
soup.find("header").attrs

In [ ]:
# That is why the line below won't work
soup.find("header", attrs={"role": "banner"})

# But this will:
# soup.find('header', attrs={'id':'globalheader'})

This matched it! We could then continue further processing by tacking on other commands:

In [ ]:
soup.find("header", attrs={"id": "globalheader"}).find_all(
    "li"
)  # li stands for list items

This returns all of our list items, and since it's within a particular header section of the page, it appears they are links to menu items for navigating the webpage. If we wanted to grab just the links within these:

In [ ]:
menu_links = set()
for list_item in soup.find("header", attrs={"id": "globalheader"}).find_all("li"):
    for link in list_item.find_all("a", href=True):
        menu_links.add(link)
menu_links  # a unique set of all the seemingly important links in the header

### Takeaway lesson <a class="anchor" id="section3_2"></a>
The above tutorial isn't meant to be a study guide to memorize; its point is to show you the most important functionaity that exist within BeautifulSoup, and to illustrate how one can access different pieces of content. No two web scraping tasks are identical, so it's useful to play around with code and try different things, while using the above as examples of how you may navigate between different tags and properties of a page. Don't worry; we are always here to help when you get stuck!

## String formatting <a class="anchor" id="section4"></a>
As we parse webpages, we may often want to further adjust and format the text to a certain way.

For example, say we wanted to scrape a polical website that lists all US Senators' name and office phone number. We may want to store information for each senator in a dictionary. All senators' information may be stored in a list. Thus, we'd have a list of dictionaries. Below, we will initialize such a list of dictionary (it has only 3 senators, for illustrative purposes, but imagine it contains many more).

In [ ]:
# this is a bit clumsy of an initialization, but we spell it out this way for clarity purposes
# NOTE: imagine the dictionary were constructed in a more organic manner
senator1 = {"name": "Lamar Alexander", "number": "555-229-2812"}
senator2 = {"name": "Tammy Baldwin", "number": "555-922-8393"}
senator3 = {"name": "John Barrasso", "number": "555-827-2281"}
senators = [senator1, senator2, senator3]
print(senators)

In the real-world, we may not want the final form of our information to be in a Python dictionary; rather, we may need to send an email to people in our mailing list, urging them to call their senators. If we have a templated format in mind, we can do the following:

In [ ]:
email_template = """Please call {name} at {number}"""
for senator in senators:
    print(email_template.format(**senator))

**Please [visit here](https://docs.python.org/3/library/stdtypes.html#str.format)** for further documentation

Alternatively, one can also format their text via the ``f'-strings`` property. [See documentation here](https://docs.python.org/3/reference/lexical_analysis.html#f-strings). For example, using the above data structure and goal, one could yield identical results via:

In [ ]:
for senator in senators:
    print(f"Please call {senator['name']} at {senator['number']}")

Additionally, sometimes we wish to search large strings of text. If we wish to find all occurrences within a given string, a very mechanical, procedural way of doing it would be to use the ``.find()`` function in Python and to repeatedly update the starting index from which we are looking.

### Regular Expressions
A way more suitable and powerful way is to use Regular Expressions, which is a pattern matching mechanism used throughout Computer Science and programming (it's not just specific to Python). A tutorial on Regular Expressions (aka regex) is beond this lab, but below are many great resources that we recommend, if you are interested in them (could be very useful for a homework problem):
- https://docs.python.org/3.3/library/re.html
- https://regexone.com
- https://docs.python.org/3/howto/regex.html.

## Walkthrough Example of web scraping  <a class="anchor" id="section5"></a>
We're going to see the structure of Project Gutenberg's best books list (**NOTE: We'll use a similar structure that is described a little more within the other Lab2_More_Pandas.ipynb notebook using data from Goodreads)**. We'll use the Developer tools in chrome, safari and firefox have similar tools available. To get this page we use the `requests` module. But first we should check if the company's policy allows scraping. Check the [robots.txt](https://www.gutenberg.org/policy/robot_access.html) to find what sites/elements are not accessible. Please read and verify.

In [ ]:
url = "https://www.npr.org/2018/11/05/664395755/what-if-the-polls-are-wrong-again-4-scenarios-for-what-might-happen-in-the-elect"
response = requests.get(url)
# response.status_code
# response.content

# Beautiful Soup (library) time!
soup = BeautifulSoup(response.content, "html.parser")
# print(soup)
# soup.prettify()
soup.find("title")

# Q1: how do we get the title's text?

# Q2: how do we get the webpage's entire content?

In [ ]:
URLSTART = "https://www.gutenberg.org"
BESTBOOKS = "/ebooks/bookshelf/13?start_index="
url = URLSTART + BESTBOOKS + "1"
print(url)
page = requests.get(url)

We can see properties of the page. Most relevant are `status_code` and `text`. The former tells us  if the web-page was found, and if found, ok.

In [ ]:
page.status_code  # 200 is good

In [ ]:
page.text[:5000]

Let us write a loop to fetch 2 pages of "Best Books Ever Listings" from Project Gutenberg. Notice the use of a format string.

In [ ]:
URLSTART = "https://www.gutenberg.org"
BESTBOOKS = "/ebooks/bookshelf/13?start_index="
for i, gbindex in enumerate([1, 26]):
    bookindex = str(gbindex)
    stuff = requests.get(URLSTART + BESTBOOKS + bookindex)
    filetowrite = f"files/page{(i+1):02d}.html"
    os.makedirs(os.path.dirname(filetowrite), exist_ok=True)
    print("FTW", filetowrite)
    fd = open(filetowrite, "w+", encoding="utf-8")
    fd.write(stuff.text)
    fd.close()
    time.sleep(2)

### Step 1. Parse the page, extract book urls

Notice how we do file input-output, and use beautiful soup in the code below. The `with` construct ensures that the file being read is closed, something we do explicitly for the file being written. We look for the elements with class `bookTitle`, extract the urls, and write them into a file

In [ ]:
bookdict = {}
for i in range(1, 3):
    books = []
    stri = "%02d" % i
    filetoread = "files/page" + stri + ".html"
    print("FTW", filetoread)
    with open(filetoread) as fdr:
        data = fdr.read()
    soup = BeautifulSoup(data, "html.parser")
    for e in soup.select(".booklink"):
        books.append(e.select_one("a")["href"])
    print(books[:10])
    bookdict[stri] = books
    fd = open("files/list" + stri + ".txt", "w")
    fd.write("\n".join(books))
    fd.close()

### Step 2. Parse a book page, extract book properties

Ok so now lets dive in and get one of these these files and parse them.

In [ ]:
furl = URLSTART + bookdict["01"][0]
furl

In [ ]:
fstuff = requests.get(furl)
print(fstuff.status_code)

In [ ]:
# d=BeautifulSoup(fstuff.text, 'html.parser')
# try this to take care of arabic strings
d = BeautifulSoup(fstuff.text, "html.parser")

In [ ]:
d.select_one("meta[property='og:title']")["content"]

In [ ]:
Image(d.select_one("meta[property='og:image']")["content"])

Lets get everything we want... Note the use of `pprint` for prettier print formatting (hence the name!).

In [ ]:
from pprint import pprint

# d=BeautifulSoup(fstuff.text, 'html.parser', from_encoding="utf-8")
pprint(
    {
        "title": d.select('[itemprop="headline"]')[0].get_text().strip(),
        "author": d.select('[itemprop="creator"]')[0].get_text().strip(),
        "cover": d.select_one("meta[property='og:image']")["content"],
    }
)

Ok, now that we know what to do, lets wrap our fetching into a proper script. So that we dont overwhelm their servers, we will only fetch 5 from each page, but you get the idea...

We'll segue of a bit to explore new style format strings. See https://pyformat.info for more info.

In [ ]:
"list{:0>2}.txt".format(3)

In [ ]:
a = "4"
b = 4


class Four:
    def __str__(self):
        return "Fourteen"


c = Four()

In [ ]:
"The hazy cat jumped over the {} and {} and {}".format(a, b, c)

### Step 3. Set up a pipeline for fetching and parsing

Ok lets get back to the fetching...

In [ ]:
fetched = []
for i in range(1, 3):
    with open("files/list{:0>2}.txt".format(i)) as fd:
        for j, bookurl_line in enumerate(fd):
            if j > 4:
                break
            bookurl = bookurl_line.strip()
            stuff = requests.get(URLSTART + bookurl)
            filetowrite = f"files/page{i:02d}_book{(j+1):02d}.html"
            print("FTW", filetowrite)
            fd = open(filetowrite, "w", encoding="utf-8")
            fd.write(stuff.text)
            fd.close()
            fetched.append(filetowrite)
            time.sleep(1)
pprint(fetched)

### <div class="exercise"><b>Exercise</b></div>
Your job is to fill in the code to get the genres. Go to your browser, enter the url of the bookpage above and "View Page Source". In the main web-page, search for the text which contains the year or the genre (or whichever field it is that you are looking for) and right-click and "Inspect Element". What properties do all the subjectes share? Use that information to identify them automatically in BeautifulSoup. Hint: CSS selectors include also the "^=" operator, which filters those tags that start with a certain substring.

In [ ]:
def get_genres(d):
    # your code here
    genres = d.select('[href^="/ebooks/subject"]')
    glist = []
    for g in genres:
        glist.append(g.get_text().strip())
    return glist


with open(fetched[0]) as fd:
    datext = fd.read()
    d = BeautifulSoup(datext, "html.parser")
    pprint(get_genres(d))

In [ ]:
listofdicts = []
for filetoread in fetched:
    print(filetoread)
    td = {}
    with open(filetoread, encoding="utf-8") as fd:
        datext = fd.read()
    d = BeautifulSoup(datext, "html.parser")
    td["title"] = d.select('[itemprop="headline"]')[0].get_text().strip()
    td["author"] = d.select('[itemprop="creator"]')[0].get_text().strip()
    td["cover"] = d.select_one("meta[property='og:image']")["content"]
    td["file"] = filetoread
    glist = get_genres(d)
    td["genres"] = "|".join(glist)
    listofdicts.append(td)

In [ ]:
listofdicts[0]

Finally lets write all this stuff into a csv file which we will use to do analysis.

In [ ]:
df = pd.DataFrame.from_records(listofdicts)
df

In [ ]:
df.to_csv("files/meta_utf8_EK.csv", index=False, header=True)

<a id="3"></a>
## Let's Learn How To Utilize APIs <a class="anchor" id="chapter6"></a>

Information below has been cited and edited from [Dataquest - Python API Tutorial](https://www.dataquest.io/blog/python-api-tutorial/)

APIs can help you retrieve data for your data science projects. APIs are the industry standard way of providing access to data. Most government websites allow utilization of their data over their APIs. This would be very handy say, if you are searching for COVID statistics day-by-day in India, right?

However, one might ask why use an API? I can download my CSVs just fine.. That could be true. But APIs are very useful in the following cases:
* If your data is changing quickly and continiously.
* You would like to access a certain part of the database and not the entire thing.
* There is repeated computation involved.
    * For example, Spotify has an API that can tell you the piece of music. You can create you own classifier to compute music categories yourself however you'll never have much data as Spotify does.

### Then, What is an API? <a class="anchor" id="section6_1"></a>

from IPython.display import ImageAn API, in other words, an Application Programming Interface is a server that you can use to retrieve and send data to using code. APIs are most commonly used to retrieve data, which by the end of this tutorial you will be proficient at.

When we would like to receive data from an API, we need to make a request. Requests are used all over the web. For instance, when you visit a blog post, your web browser makes a request to a web server, which responds with the content of the web page.

In [ ]:
Image("figs/api.png")

API works in the same way. You make a request to an API server for data, and you get a response.


### Making API Requests in Python <a class="anchor" id="section6_2"></a>

To work with APIs in Python we can use the most common **requests** library.
<div class="alert alert-block alert-info">
<b>Note:</b> As it is not part of base Python you will need to install it through pip install with the following: 'pip install requests' .
</div>



### Our First API Request <a class="anchor" id="section6_3"></a>

There are many different types of requests. The most commonly used one, a GET request, is used to retrieve data. Because we’ll just be working with retrieving data, our focus will be on making ‘get’ requests. When we make a request, the response from the API comes with a response code which tells us whether our request was successful. Response codes are important because they immediately tell us if something went wrong.


But first, let us find an end point to connect to. We will use [Open Governence Data Platform India](https://data.gov.in/) for this endevour which could be a good example for your projects. Open Government Data Platform India or *data.gov.in* is a platform for supporting Open data initiative of Government of India. This portal is a single-point access to datasets, documents, services, tools and applications published by ministries, departments and organisations of the Government of India. You will now be introduced to how to use this website as an example for your future researches. Other end points are laid out in a similar matter with different documentation and structure which you must research and try to use yourself.

Let us start our example. The website itself looks like this:


In [ ]:
Image("figs/datagovin.png", width=1400, height=1000)

To utilize this website let us first find a subject through the catalog. You can also select certain domains (databases) and sectors as seen below.

In [ ]:
Image("figs/datagovin2.png")

I arbitrarily chose covid as a topic and it has listed me all the APIs regarding the matter.

In [ ]:
Image("figs/datagovin3.png")

After I select a API I am interested in, this page pops up where it asks for certain parameters.
* API Key: This is specific to this website in particular, however do expect many other databases to establish their own rules.
    * In this website, you need to create an account in order to receive a member specific key which you can use to get the entirety of the API output.
    * With a sample key you can only access 10 data points.
* Format : Here, many are tempted to get the output format as CSV however it is easier to just get a JSON file for now and only after then turn it into a csv which we will do later on.
* Offset : If you would like to skip some records.
* Limit : Maximum number of records.
* Filters : Filters the results with respect to fields specified.

There is no need to input all the parameters, the critical ones are the API key and format.

In [ ]:
Image("figs/datagovin5.png")

In [ ]:
Image("figs/datagovin6.png")

To make a ‘GET’ request, we’ll use the requests.get() function, which requires one argument — the URL we want to make the request to. Please note that we copied the request url in the image above on the url part in the code below. When you have your own api it is important that you change the code after the part "<font color = red>?api-key= </font>"

In [ ]:
import requests
import pandas as pd
import json  # We can use JSON operations with this, we need it to change it a pandas dataframe later on.
from pandas import (
    json_normalize,
)  # we use this so that we can stop an error such as arrays being not in the same length in fields in the json file.


resp = requests.get(
    "https://api.data.gov.in/resource/0bdce1aa-10a9-4dcd-abfd-33ba51a879d6?api-key=579b464db66ec23bdd000001cdd3946e44ce4aad7209ff7b23ac571b&format=json"
)

The <font color = 'lightblue'> get()</font> function returns a <font color = "orange">response </font> object. We can use the response.status_code attribute to receive the status code for our request:

In [ ]:
print(resp.status_code)

<div class="alert alert-block alert-success">
We received a 200! This means our request was succesful. .
</div>

There are many API Status Codes. You can find a small snippet of them here if you are interested, and are willing to understand what went wrong in the future. Cited from [Dataquest - Python API Tutorial](https://www.dataquest.io/blog/python-api-tutorial/)

In [ ]:
Image("figs/status.png")

Now that we have received a response and stored it in our "response" object, let us read what is inside the file as a JSON.

In [ ]:
print(resp.json())

As you can see very hard to utilize!

Here you have 2 options you can choose. Either work with JSON which you can read further in the link provided in [Part 3: Let's Learn How To Utilize APIs](#3) or turn this data into a pandas dataframe. The continuation of this introduction will focus on the pandas dataframe route which was summarized from [this post in Stackoverflow](https://stackoverflow.com/questions/1871524/how-can-i-convert-json-to-csv).

In [ ]:
content = json.loads(
    resp.content
)  # We find the content of the response variable and we load it utilizing json.loads into a variable.

In [ ]:
content  # The variable shows what it has inside as content. Now here please be careful as we are looking for things that resemble a data cluster.
# In this example we can see that the data itself starts after the 'records' and thus must be kept in mind.

In [ ]:
"""
As we realized that the 'records' were where the data was starting to be written, we used that
to create a data frame from the dictionary that was inside the content of the response. 
"""

df = pd.DataFrame.from_dict(content["records"])

In [ ]:
df.head()

There we go! We have the data now as a pandas data frame. From this moment on we can do any operations that we deem necessary. Follow the course to learn how :)

You can also save this data frame in a csv file just like this:

In [ ]:
df.to_csv("sample.csv")

<div class="alert alert-block alert-warning">
<b>Remember! </b> Always try to research the web first when you encounter a block - be it a bug, or a general sense of not knowing what to do. This will make you into better scientists. Hey, even I studied a bit on the web to remember how to do this! You can do it too :)
</div>

<a id = "cite"></a>
**Citations utilized:**

[ValueError: arrays must all be same length](https://stackoverflow.com/questions/50531308/valueerror-arrays-must-all-be-same-length)

[Dataquest - Python API Tutorial](https://www.dataquest.io/blog/python-api-tutorial/)


Best of Luck with your studies!